In [54]:
import cv2
import mediapipe as mp
import math

mp_hands=mp.solutions.hands
hands=mp_hands.Hands()
mp_hands.Hands(
    static_image_mode=False,                # 影片流或靜態圖像，先檢測後追蹤
    # 手繪地標模型的複雜性: 0 或 1。地標精度以及推斷延遲通常與模型複雜性有關。默認 1。
    model_complexity=0,
    # max_num_hands=1,                      # 最大手數量
    min_detection_confidence=0.5,           # 檢測閥值
        min_tracking_confidence=0.5)
mp_drawing=mp.solutions.drawing_utils#畫圖方法
hands_LmsStyle=mp_drawing.DrawingSpec(color=(0,0,255),thickness=5)
hands_ConStyle=mp_drawing.DrawingSpec(color=(0,255,0),thickness=10)

In [55]:
# 根據兩點的座標，計算角度
def vector_2d_angle(v1, v2):
    v1_x = v1[0]
    v1_y = v1[1]
    v2_x = v2[0]
    v2_y = v2[1]
    try:
        angle_ = math.degrees(math.acos(
            (v1_x*v2_x+v1_y*v2_y)/(((v1_x**2+v1_y**2)**0.5)*((v2_x**2+v2_y**2)**0.5))))
    except:
        angle_ = 180
    return angle_

In [56]:
# 根據傳入的 21 個節點座標，得到該手指的角度
def hand_angle(hand_):
    angle_list = []

    angle_ = vector_2d_angle(
        ((int(hand_[0][0]) - int(hand_[2][0])),
         (int(hand_[0][1])-int(hand_[2][1]))),
        ((int(hand_[3][0]) - int(hand_[4][0])),
         (int(hand_[3][1]) - int(hand_[4][1])))
    )
    angle_list.append(angle_)

    angle_ = vector_2d_angle(
        ((int(hand_[0][0])-int(hand_[6][0])),
         (int(hand_[0][1]) - int(hand_[6][1]))),
        ((int(hand_[7][0]) - int(hand_[8][0])),
         (int(hand_[7][1]) - int(hand_[8][1])))
    )
    angle_list.append(angle_)

    angle_ = vector_2d_angle(
        ((int(hand_[0][0]) - int(hand_[10][0])),
         (int(hand_[0][1]) - int(hand_[10][1]))),
        ((int(hand_[11][0]) - int(hand_[12][0])),
         (int(hand_[11][1]) - int(hand_[12][1])))
    )
    angle_list.append(angle_)

    angle_ = vector_2d_angle(
        ((int(hand_[0][0]) - int(hand_[14][0])),
         (int(hand_[0][1]) - int(hand_[14][1]))),
        ((int(hand_[15][0]) - int(hand_[16][0])),
         (int(hand_[15][1]) - int(hand_[16][1])))
    )
    angle_list.append(angle_)

    angle_ = vector_2d_angle(
        ((int(hand_[0][0]) - int(hand_[18][0])),
         (int(hand_[0][1]) - int(hand_[18][1]))),
        ((int(hand_[19][0]) - int(hand_[20][0])),
         (int(hand_[19][1]) - int(hand_[20][1])))
    )
    angle_list.append(angle_)
    return angle_list

In [57]:
# 根據手指角度的串列內容，返回對應的手勢名稱
def hand_pos(finger_angle):
    f1 = finger_angle[0]
    f2 = finger_angle[1]
    f3 = finger_angle[2]
    f4 = finger_angle[3]
    f5 = finger_angle[4]

    if f1 <= 50 and f2 >= 50 and f3 >= 50 and f4 >= 50 and f5 >= 50:
        return 'GOOD'
    elif f1 >= 50 and f2 < 50 and f3 <= 50 and f4 >= 50 and f5 >= 50:
        return 'YA'
    elif f1 <= 50 and f2 < 50 and f3 > 50 and f4 >= 50 and f5 <= 50:
        return 'ROCK'
    else:
        return ''

In [58]:
cap = cv2.VideoCapture(0) # 選擇webcan鏡頭

while True:
    ret,img=cap.read() # ret為是否開啟鏡頭 img為每個frame
    img_rgb=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results=hands.process(img_rgb)
    h=img_rgb.shape[0]
    w=img_rgb.shape[1]
    if results.multi_hand_landmarks:
           # print(results.multi_hand_landmarks)
            #break
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(img, hand_landmarks, mp_hands.HAND_CONNECTIONS, hands_LmsStyle, hands_ConStyle)
                finger_points=[]
                for i in hand_landmarks.landmark:
                    
                    x=i.x*w
                    y=i.y*h
                    finger_points.append((x,y))
                if finger_points:
                    finger_angle=hand_angle(finger_points)
                    text=hand_pos(finger_angle)
                    cv2.putText(img,text,(30,120),cv2.FONT_HERSHEY_SIMPLEX,3,(255,0,0),10)
    
    cv2.imshow('hand',img) #  開啟視窗 視窗名稱為hand 影像為img
    if cv2.waitKey(5)== ord('q'): # 當當我今天按下英文字母q 會做跳出的動作
        break
cap.release
cv2.destroyAllWindows() 


KeyboardInterrupt: 